# read files

In [1]:
import pymssql
import pymysql as mysql
import os
import re
import pandas as pd
import datetime

In [11]:
class stt_server:

    def __init__(self, cpu_id):

        # settings ++
        self.cpu_id = cpu_id
        self.cpu_cores = [i for i in range(0,15)]

        # ms sql
        self.sql_name = 'voice_ai'
        self.sql_server = '10.2.4.124'
        self.sql_login = 'ICECORP\\1c_sql'

        # mysql
        self.mysql_name = {
            1: 'MICO_96',
            2: 'asterisk',
        }
        self.mysql_server = '10.2.4.146'
        self.mysql_login = 'asterisk'

        self.script_path = '/home/alex/projects/call_centre_stt_server/'
        self.model_path = '/home/alex/projects/vosk-api/python/example/model'
        self.source_id = 0
        self.sources = {
            'call': 1,
            'master': 2,
        }
        self.original_storage_path = {
            #1: '/mnt/share/audio_call/',
            #2: '/mnt/share/audio_master/REC_IN_OUT/',
            1: '/mnt/share/audio/MSK_SRVCALL/RX_TX/',
            2: '/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/'
        }
        """self.original_storage_prefix = {
            1: 'RXTX_',
            2: '',
        }"""
        self.temp_file_path = self.script_path+'files/'
        # settings --

        self.temp_file_name = ''
        self.original_file_path = ''
        #self.original_file_name = ''
        self.original_file_duration	= 0
        self.date_y = ''
        self.date_m = ''
        self.date_d = ''
        self.rec_date = ''

        #store pass in file, to prevent pass publication on gitdelete_current_queue
        with open(self.script_path+'sql.pass','r') as file:
            self.sql_pass = file.read().replace('\n', '')
            file.close()

        with open(self.script_path+'mysql.pass','r') as file:
            self.mysql_pass = file.read().replace('\n', '')
            file.close()

        self.conn = self.connect_sql()
        self.mysql_conn = {
            1: self.connect_mysql(1),
            2: self.connect_mysql(2),
        }

    def connect_sql(self):

        return pymssql.connect(
            server = self.sql_server,
            user = self.sql_login,
            password = self.sql_pass,
            database = self.sql_name,
            #autocommit=True
        )

    def connect_mysql(self, source_id):

        return mysql.connect(
            host = self.mysql_server, 
            user = self.mysql_login, 
            passwd = self.mysql_pass,
            db = self.mysql_name[source_id],
            #autocommit = True
        )

    def get_fs_files_list(self, queue):
        
        fd_list = []

        if self.source_id == self.sources['call']:
            #print('call', len(queue), self.original_file_path)
            for root, dirs, files in os.walk(self.original_storage_path[self.source_id]):
                #find_files += [os.path.join(root, name) for name in files if name[-4:] == '.wav'] 
                for filename in files:
                    if filename[-4:] == '.wav' and not filename in queue:
                        rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)
                        if len(rec_source_date) and len(rec_source_date[0]):
                            rec_date = rec_source_date[0][:10] + ' ' + rec_source_date[0][11:].replace('-', ':')

                            if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                                rec_date = 'Null'
                                print('0 Unable to extract date:', root, filename)

                            date_string  = re.findall(r'\d{4}-\d{2}-\d{2}', filename)
                            if len(date_string):
                                self.date_y = date_string[0][:4]
                                self.date_m = date_string[0][5:-3]
                                self.date_d = date_string[0][-2:]
                                #print(date_y, date_m, date_d, filename)
                                linkedid, dst, src = self.linkedid_by_filename(filename) # cycled query

                                fd_list.append({
                                    'filepath': root,
                                    'filename': filename,
                                    'rec_date': rec_date,
                                    'src': src,
                                    'dst': dst,
                                    'linkedid': linkedid,
                                })
                        else:
                            print('1 Unable to extract date:', root, filename)
        
        elif self.source_id == self.sources['master']:
            files_list = []
            for (dirpath, dirnames, filenames) in os.walk(self.original_storage_path[self.source_id]):
                files_list.extend(filenames)
                break

            # get record date
            for filename in files_list:
                if not filename in queue:
                    rec_date = 'Null'

                    #elif self.source_id == self.sources['master']:
                    uniqueid = re.findall(r'^\d*.\d*', filename)[0]
                    cursor = self.mysql_conn[self.source_id].cursor()
                    query = "select calldate, src, dst from cdr where uniqueid = '" + uniqueid + "' limit 1;"
                    cursor.execute(query)  # cycled query
                    src = ''
                    dst = ''
                    linkedid = uniqueid
                    for row in cursor.fetchall():
                        rec_date = str(row[0])
                        src = str(row[1])
                        dst = str(row[2])
                    if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                        rec_date = 'Null'
                        print('Unable to extract date from filename', filename)

                    fd_list.append({
                        'filepath': self.original_storage_path[self.source_id],
                        'filename': filename,
                        'rec_date': rec_date,
                        'src': src,
                        'dst': dst,
                        'linkedid': linkedid,
                    })

        df = pd.DataFrame(fd_list, columns = ['filepath', 'filename', 'rec_date', 'src', 'dst', 'linkedid'])
        df.sort_values(['rec_date', 'filename'], ascending=True, inplace=True)

        return df.values
    
    def get_sql_complete_files(self):

        cursor = self.conn.cursor()
        sql_query = "select distinct filename from queue where"
        sql_query += " source_id='" + str(self.source_id) + "'"
        sql_query += " order by filename;"
        cursor.execute(sql_query)
        complete_files = []
        for row in cursor.fetchall():
            complete_files.append(row[0])

        return complete_files
    
    def get_source_id(self, source_name):
        for source in self.sources.items():
            if source[0] == source_name:
                return source[1]
        return 0
    
    def set_today_ymd(self):
        self.date_y	= datetime.datetime.today().strftime('%Y')
        self.date_m	= datetime.datetime.today().strftime('%m')
        self.date_d	= datetime.datetime.today().strftime('%d')
        
    def linkedid_by_filename(self, original_file_name):

        filename = original_file_name.replace('rxtx.wav', '')

        
        #print('linkedid_by_filename', self.date_y, self.date_m, self.date_d, original_file_name)
        
        date_from = datetime.datetime(int(self.date_y), int(self.date_m), int(self.date_d))
        date_toto = date_from+datetime.timedelta(days=1)
        date_from = datetime.datetime.strptime(str(date_from), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')
        date_toto = datetime.datetime.strptime(str(date_toto), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')

        mysql_conn = self.connect_mysql(self.source_id)

        with mysql_conn:
        #with self.mysql_conn[self.source_id]:
            query = """
            select 
                linkedid,
                SUBSTRING(dstchannel, 5, 4),
                src
                from PT1C_cdr_MICO as PT1C_cdr_MICO
                where 
                    calldate>'"""+date_from+"""' and 
                    calldate<'"""+date_toto+"""' and 
                    PT1C_cdr_MICO.recordingfile LIKE '%"""+filename+"""%' 
                    limit 1;"""

            #cursor = mysql_conn[self.source_id].cursor()
            cursor = mysql_conn.cursor()
            cursor.execute(query)
            for row in cursor.fetchall():
                linkedid, dstchannel, src = row[0], row[1], row[2]
                #print('linkedid, dstchannel', linkedid, dstchannel)
                return linkedid, dstchannel, src
        return '', '', ''

filename = 'in_74954825630_2020-11-25-21-54-00rxtx.wav'
date_string  = re.findall(r'\d{4}-\d{2}-\d{2}', filename)
if len(date_string):
    date_y = date_string[0][:4]
    date_m = date_string[0][5:-3]
    date_d = date_string[0][-2:]
date_y, date_m, date_d

### file list 1

In [3]:
server_object = stt_server(0)
server_object.source_id = 1

In [4]:
complete_files = server_object.get_sql_complete_files()
len(complete_files)

450

In [5]:
new_files = server_object.get_fs_files_list(complete_files)
len(new_files)

14240

In [6]:
new_files[0]

array(['/mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2020-11/25',
       'in_9670330468_2020-11-25-21-24-49rxtx.wav', '2020-11-25 21:24:49',
       '9670330468', '5023', '1606328688.15369803'], dtype=object)

### file list 2

In [12]:
server_object = stt_server(0)
server_object.source_id = 2

In [13]:
complete_files_2 = server_object.get_sql_complete_files()
len(complete_files_2)

2224

In [14]:
new_files_2 = server_object.get_fs_files_list(complete_files_2)
len(new_files_2)

Unable to extract date from filename 1613546579.1710346-in.wav
Unable to extract date from filename 1613546579.1710346-out.wav
Unable to extract date from filename 1613546586.1710350-in.wav
Unable to extract date from filename 1613546586.1710350-out.wav
Unable to extract date from filename 1613546598.1710369-in.wav
Unable to extract date from filename 1613546598.1710369-out.wav
Unable to extract date from filename 1613546607.1710381-in.wav
Unable to extract date from filename 1613546607.1710381-out.wav
Unable to extract date from filename 1613546609.1710385-in.wav
Unable to extract date from filename 1613546609.1710385-out.wav
Unable to extract date from filename 1613546637.1710403-in.wav
Unable to extract date from filename 1613546637.1710403-out.wav
Unable to extract date from filename 1613546644.1710407-in.wav
Unable to extract date from filename 1613546644.1710407-out.wav
Unable to extract date from filename 1613546654.1710421-in.wav
Unable to extract date from filename 1613546654.

43711

In [15]:
new_files_2[0]

array(['/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/',
       '1612424800.1439441-in.wav', '2021-02-04 10:46:40', '9647664004',
       '989859259303', '1612424800.1439441'], dtype=object)

### next

In [32]:
def find_files(catalog):
    fd_list = []
    for root, dirs, files in os.walk(catalog):        
        #find_files += [os.path.join(root, name) for name in files if name[-4:] == '.wav'] 
        for filename in files:
            if filename[-4:] == '.wav':
                #print(filename)
                rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)
                if len(rec_source_date) and len(rec_source_date[0]):
                    rec_date = rec_source_date[0][:10] + ' ' + rec_source_date[0][11:].replace('-', ':')
                    
                    if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                        rec_date = 'Null'
                        print('0 Unable to extract date:', root, filename)

                    fd_list.append({
                        'filepath': root,
                        'filename': filename,
                        'rec_date': rec_date
                    })
                else:
                    print('1 Unable to extract date:', root, filename)
    return fd_list

In [33]:
fs_list = find_files(server_object.original_storage_path[server_object.source_id])
len(fs_list), fs_list[0]

(13398,
 {'filepath': '/mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2020-11/25',
  'filename': 'in_74954825630_2020-11-25-21-54-00rxtx.wav',
  'rec_date': '2020-11-25 21:54:00'})

### remove complete files

In [32]:
transcribed = []
server_object = stt_server(0)
server_object.source_id = 1
cursor = server_object.conn.cursor()
sql_query = "select distinct audio_file_name as filename, date_y, date_m, date_d"
sql_query += " from transcribations where source_id=1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    transcribed.append({
                'filename': row[0],
                'date_y': row[1],
                'date_m': row[2],
                'date_d': row[3],
            })
len(transcribed), transcribed[0]

(279736,
 {'filename': 'in_4953894961_2021-01-13-19-00-39rxtx.wav',
  'date_y': '2021',
  'date_m': '01',
  'date_d': '13'})

In [37]:
full_path = server_object.original_storage_path[1]
full_path += 'RXTX_' + transcribed[0]['date_y']
full_path += '-' + transcribed[0]['date_m'] + '/'
full_path += transcribed[0]['date_d'] + '/'
full_path += transcribed[0]['filename']
os.path.isfile(full_path), full_path

(True,
 '/mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2021-01/13/in_4953894961_2021-01-13-19-00-39rxtx.wav')

# end

In [ ]:
server_object = stt_server(0)
server_object.set_today_ymd()
for source_id in server_object.sources: # ['call', 'master']
    #server_object.source_id = server_object.sources['call']
    server_object.source_id = server_object.get_source_id(source_id)
    complete_files	= server_object.get_sql_complete_files()
    incomplete_count = 0
    complete_count = 0
    print('server_object.source_id', server_object.source_id)
    for filename, rec_date in server_object.get_fs_files_list():
        print(filename, rec_date)
    break

In [ ]:
complete_files

In [20]:
server_object = stt_server(0)
server_object.source_id = 2
fd_list = server_object.get_fs_files_list()
len(fd_list)

44452

In [17]:
fd_list[0]

array(['2021-02-04 10:46:40', '1612424800.1439441-in.wav'], dtype=object)

In [ ]:
df = pd.DataFrame(fd_list)
df.sort_values(['rec_date', 'filename'], ascending=False, inplace=True)

In [ ]:
#df[df.rec_date=='Null']
for f,d in df.values:
    print(f,d)
    break

In [ ]:
filename = '1612946589.1564912-in.wav'
uniqueid = re.findall(r'^\d*.\d*', filename)[0]
uniqueid

In [44]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "select calldate from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    #rec_date = str(row[0])
    print(row[0])

2021-02-04 11:36:33


In [ ]:
def get_source_id(source_name):
    for source in server_object.sources.items():
        if source[0] == source_name:
            return source[1]
    return 0
get_source_id('master')

In [ ]:
server_object.rec_date = 'Null'
filename = 'in_4957237230_2021-02-10-07-16-03rxtx.wav'
rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)[0]                
if len(rec_source_date):
    server_object.rec_date = rec_source_date[:10] + ' ' + rec_source_date[11:].replace('-', ':')
if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', server_object.rec_date)) == 0:
    print('Unable to extract date from filename', filename)
server_object.rec_date

In [ ]:
rec_source_date

In [ ]:
server_object.rec_date

In [ ]:
re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', server_object.rec_date)

In [ ]:
for source_id in server_object.sources:
    print(get_source_id(source_id))

In [ ]:
server_object = stt_server(0)
server_object.source_id = 1

In [ ]:
files = server_object.get_fs_files_list()
len(files), files[0]

In [ ]:
files

In [ ]:
complete = server_object.get_sql_complete_files()
len(complete)

### transcribations show columns

In [37]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('transcribation_date',)
('date_y',)
('date_m',)
('date_d',)
('side',)
('text',)
('start',)
('audio_file_name',)
('conf',)
('end_time',)
('sentiment',)
('sentiment_pos',)
('sentiment_neg',)
('ID',)
('linkedid',)
('dst',)
('record_date',)
('source_id',)
('src',)


### check date cdr

In [48]:
datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2021-02-11 12:16:12'

In [46]:
uniqueid = '1613032036.1593933'

In [47]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "select calldate from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    #rec_date = str(row[0])
    print(row[0])

2021-02-11 11:27:16


### PT1C_cdr_MICO show columns

In [14]:
cursor = server_object.mysql_conn[1].cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='PT1C_cdr_MICO';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('id',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('channel',)
('dstchannel',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('answer',)
('end',)
('recordingfile',)
('peeraccount',)
('linkedid',)
('sequence',)


### queue show columns

In [9]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('date',)
('cpu_id',)
('filepath',)
('date_y',)
('date_m',)
('date_d',)
('filename',)
('duration',)
('record_date',)
('source_id',)


### transcribations show columns

In [12]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('transcribation_date',)
('date_y',)
('date_m',)
('date_d',)
('side',)
('text',)
('start',)
('audio_file_name',)
('conf',)
('end_time',)
('sentiment',)
('sentiment_pos',)
('sentiment_neg',)
('ID',)
('linkedid',)
('dst',)
('record_date',)
('source_id',)
('src',)


### get record date from cdr

In [6]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='cdr';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('did',)
('recordingfile',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('did',)
('recordingfile',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('userfield',)
('did',)
('recordingfile',)
('id',)
('calldate',)
('clid',)
('src',)
('dst',)
('dcontext',)
('channel',)
('dstchannel',)
('lastapp',)
('lastdata',)
('duration',)
('billsec',)
('disposition',)
('amaflags',)
('accountcode',)
('uniqueid',)
('peeraccount',)
('linkedid',)
('sequence',)
('userfield',)

In [12]:
server_object.source_id = 2
#server_object.original_file_name = '1612427793.1440877-in.wav'#files[0] #debug
server_object.original_file_name = '1612427793.1440877-in.wav'#files[0] #debug
uniqueid = re.findall(r'^\d*.\d*', server_object.original_file_name)[0]
print('uniqueid', uniqueid)
cursor = server_object.mysql_conn[server_object.source_id].cursor()
#query = "select src, dst from cdr where uniqueid = '"+uniqueid+"' limit 1;"
query = "select calldate, src, dst from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

uniqueid 1612427793.1440877
(datetime.datetime(2021, 2, 4, 11, 36, 33), '5113', '989533444853')


### insert into queue debug

In [100]:
server_object = stt_server(0)
sql_query = "insert into transcribations( audio_file_name, transcribation_date, date_y, date_m, date_d, text, start, end_time, side, conf, linkedid, src, dst, record_date, source_id) values ( '1613048825.1603082-in.wav', '2021-02-11T16:51:28', '2021', '02', '11', 'аллё здравствуйте вячеслав мареку компании единственная служба сервиса звоню по вашему резюме на избита удобно сейчас разговаривать', '6.27', '15.6', '0', '0.8597108235294117', '1613048825.1603082', '', '','None' ,'2');"
#cursor = server_object.conn.cursor()
#cursor.execute(sql_query)
#server_object.conn.commit() # autocommit

### delete from queue

In [30]:
server_object = stt_server(0)
#server_object.source_id = 1
cursor = server_object.conn.cursor()
#sql_query = "delete from queue where source_id = 2;"
sql_query = "delete from queue;"
cursor.execute(sql_query)
server_object.conn.commit() # autocommit

### select from queue

In [20]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()
#sql_query = "select count(filename) from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select distinct cpu_id from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select filename, record_date, cpu_id, duration, source_id from queue "
#sql_query += "where source_id='" + str(server_object.source_id) + "' "
#sql_query += "where duration=60 "
#sql_query += "order by record_date;"

print('# 1')
sql_query = "select top 1 * from queue "
sql_query += "where source_id='1';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)

print('# 2')
sql_query = "select top 1 * from queue "
sql_query += "where source_id='2' "
sql_query += "order by record_date;"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# count 1')
sql_query = "select count(filename) from queue where source_id = '1';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# count 2')
sql_query = "select count(filename) from queue where source_id = '2';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# cpus')
sql_query = "select distinct cpu_id from queue where cpu_id='30' order by cpu_id;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)


# 1
(datetime.datetime(2021, 2, 17, 12, 41, 47), 10, '/mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2021-02/10', None, None, None, 'in_9155948611_2021-02-10-15-23-39rxtx.wav', 0.0, datetime.datetime(2021, 2, 10, 15, 23, 39), 1, '9155948611', '5018', '1612959817.1473645')
# 2
(datetime.datetime(2021, 2, 17, 13, 8, 38), 20, '/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', None, None, None, '1612424800.1439441-in.wav', 83.24, datetime.datetime(2021, 2, 4, 10, 46, 40), 2, '9647664004', '989859259303', '1612424800.1439441')
# count 1
(15241,)
# count 2
(21946,)
# cpus
(30,)


### call queue

In [40]:
server_object = stt_server(0)
server_object.source_id = 1
cursor = server_object.conn.cursor()
#sql_query = "select top 2 * from queue where source_id='1' and duration = 0;"
sql_query = "select count(filename) from queue where source_id='2' and duration >5;"
cursor.execute(sql_query)
for row in cursor.fetchall():    
    print(row)

(41725,)


### delete from transcribations

In [49]:
cursor = server_object.conn.cursor()
sql_query = "delete from transcribations where source_id = '2';"
#cursor.execute(sql_query)
#server_object.conn.commit() # autocommit

### transcribed phrases

In [122]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()

#sql_query = "select audio_file_name, linkedid, record_date, start, side, text from transcribations where source_id = '2' order by record_date, start, side;"
sql_query = "select top 4 audio_file_name, side, record_date from transcribations where source_id = '2' group by audio_file_name, side, record_date order by record_date desc;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

('1612503468.1457247-in.wav', False, datetime.datetime(2021, 2, 5, 8, 37, 48))
('1612503468.1457247-in.wav', True, datetime.datetime(2021, 2, 5, 8, 37, 48))
('1612502951.1457175-in.wav', False, datetime.datetime(2021, 2, 5, 8, 29, 11))
('1612502951.1457175-out.wav', True, datetime.datetime(2021, 2, 5, 8, 29, 11))


In [124]:
cursor = server_object.conn.cursor()

sql_query = "select start, side, text from transcribations where linkedid = '1612502951.1457175' order by start, side;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

(0.09, True, 'и')
(6.12, True, 'новации')
(7.05, False, 'здравствуйте татьяна беспокоит скажите пожалуйста вы перераспределение присылайте сегодня да')
(9.33, True, 'здравствуй черт')
(12.69, True, 'да')
(13.92, False, 'а вы не могли бы вычесть заявки бирюкова пятьдесят девятая в конце цифр')
(19.78494, True, 'как она там по-арабски')
(21.93, False, 'ну да ну да')
(24.870641, True, 'выглядит заявок')
(27.371929, True, 'одолели дальше обсудим')
(29.04, False, 'пасибо')


In [18]:
sql_query

"select * from queue where source_id='2' limit 1;"

In [ ]:
server_object.get_sql_complete_files()

### select from transcribations

In [4]:
server_object = stt_server(0)
#server_object.source_id = 1

cursor = server_object.conn.cursor()
sql_query = "select top 1 * from transcribations;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

(datetime.datetime(2020, 10, 9, 8, 39, 8), '2020', '10', '09', True, '', 0.0, 'in_4957237230_2020-10-09-07-36-05rxtx.wav', None, None, None, None, None, 1, None, None, datetime.datetime(2020, 10, 9, 7, 36, 5), 1)


# other

In [ ]:
import datetime

In [ ]:
cur_date = datetime.datetime.now()
DD = datetime.timedelta(days=int(365 / 2))
crop_date = cur_date - DD
cur_date_y = crop_date.strftime("%Y")
cur_date_m = crop_date.strftime("%m")
cur_date_d = crop_date.strftime("%d")
cur_date_y, cur_date_m, cur_date_d

In [ ]:
cur_date = datetime.datetime.now()
cur_date

In [ ]:
DD = datetime.timedelta(days=int(365/2))
crop_date = cur_date - DD
crop_date

In [ ]:
#datetime.datetime.now().year
datetime.datetime.now().strftime("%Y")

In [ ]:
#datetime.datetime.now().month
datetime.datetime.now().strftime("%m")

In [ ]:
cur_date = datetime.datetime.strptime("2021-04-12T07:00:00Z","%Y-%m-%dT%H:%M:%SZ")
DD = datetime.timedelta(days=int(365 / 2))
crop_date = cur_date - DD
#datetime.datetime.strptime(str(datetime.datetime.now()),"%m")
crop_date

In [ ]:
#datetime.datetime.now().day
datetime.datetime.now().strftime("%d")

In [ ]:
import pandas as pd
from init_server import stt_server
from deeppavlov import build_model, configs
import numpy as np

In [ ]:
BATCH_SIZE = 3

In [ ]:
server_object = stt_server(0)

In [ ]:
query = """
    select top """+str(BATCH_SIZE)+""" 
    id,
    text,
    sentiment
    from transcribations 
    where sentiment is NULL and text!=''
    """

In [ ]:
df = pd.read_sql(query, server_object.conn)

model = build_model(configs.classifiers.rusentiment_bert, download=True) #download first time
res = model(df.text)
df['sentiment'] = model(df.text)

In [ ]:
print(df)

In [ ]:
df['sentiment'] = ['neutral','negative','skip']

In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    print(index, row.id,row.sentiment)

In [ ]:
for row in df.values:
    print(row.sentiment)

In [ ]:
for index, row in df.iterrows():
    if row.sentiment == 'negative':
        neg = 1
        pos = 0
    else:
        neg = 0
        pos = 1
    query = "update transcribations set "
    query += "sentiment = '"+row.sentiment+"', "
    query += "sentiment_neg = "+str(neg)+", "
    query += "sentiment_pos = "+str(pos)+" "
    query += "where id = "+str(row.id)
    break

In [ ]:
query

In [ ]:
df['sentiment_neg']=np.zeros(len(df))
df['sentiment_pos']=np.zeros(len(df))

In [ ]:
sentiments = pd.DataFrame(columns=['sentiment','sentiment_neg','sentiment_pos'])

In [ ]:
sentiments = sentiments.append(pd.DataFrame({
    'sentiment':
        [
            'negative',
            'positive',
            'neutral',
            'speech',
            'skip'
        ],
    'sentiment_neg':
        [
            1,0,0,0,0
        ],
    'sentiment_pos':
        [
            0,1,1,1,1
        ]
}))

In [ ]:
sentiments

In [ ]:
df

In [ ]:
df[df.sentiment=='negative'].sentiment_neg=np.ones(len(df[df.sentiment=='negative']))

In [ ]:
df[df.sentiment=='negative']

In [ ]:
df[df.sentiment=='negative'].sentiment_neg=[1]

In [ ]:
df[df.sentiment=='negative'].sentiment_neg

In [ ]:
df.set_value[df.sentiment=='negative']['sentiment_neg']=np.ones(len(df[df.sentiment=='negative']))
#df[df.sentiment!='negative']['sentiment_pos']=1
#df.fillna(0)
#df.set_value('C', 'x', 10)

In [ ]:
df[df.sentiment=='negative']['sentiment_neg']